<a href="https://colab.research.google.com/github/aso1801030/AI_teach2020/blob/master/2020AI0401_lang_character.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 異なる文字を使う弦を分類する

### ライブラリのimport


In [1]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

### 文字コードの値をインデックス流にした配列を作り、各コード値（コードポイント）別に文章中の出現頻度をカウントする関数を定義

In [2]:
# Unicode文字列の文字コード値（コードポイント）頻度測定
def count_codePoint(str):
  # Unicodeのコードポイントをインデックス位置（アドレス）とする配列を用意する
  counter = np.zeros(65535)
  # 引数strの文字列だけループ
  for i in range(len(str)):
    # 各文字列のコード値を変数に代入
    code_point = ord(str[i])
    # 万一Unicodeの範囲外なら、無視する
    if code_point > 65535:
      continue
    # if文終わり
    #　対応するアドレスの出現回数カウントをインクリメント
    counter[code_point] += 1
    # for文終わり
  # 各要素を文字列で割って正規化（出現率、つまり%にかえる）
  counter = counter/len(str)
  return counter
# 関数定義終わり


###  学習用データを作成する

In [3]:
# 学習用データの準備
ja_str = "これは日本語の文章です。"
en_str = "This is Englis Sentences."
th_str = "นี่คือภาษาไทย"
# それぞれの出現頻度割合を３つの要素として説明変数xを作る
x_train = [count_codePoint(ja_str),count_codePoint(en_str),count_codePoint(th_str)]
# 同じ順番で正解となる目的変数（ラベル）を作る
y_train = ['ja','en','th']

### 言語ごとの文字列のパターンを学習する

In [4]:
# 今回はガウス分布によるナイーブベイズアルゴリズムで学習モデルを作る
clf = GaussianNB()
clf.fit( x_train, y_train )

GaussianNB(priors=None, var_smoothing=1e-09)

### テスト検証用データを作成する

In [5]:
# テスト検証用データ（評価データ）の作成
ja_test_str = "こんにちは"
en_test_str = "Hello"
th_test_str = "สวัสดี"

# 説明変数にする
x_test = [count_codePoint(ja_test_str),count_codePoint(en_test_str),count_codePoint(th_test_str)]
# 同じ順番で正解となる目的変数を作る
y_test = ['ja','en','th']


### テスト検証（推論）し、評価する

In [6]:
# 推論させる
y_pred = clf.predict(x_test)
print("１つ目の文章の言語は",y_pred[0],"です")
print("2つ目の文章の言語は",y_pred[1],"です")
print("3つ目の文章の言語は",y_pred[2],"です")
print("正解率＝",accuracy_score(y_test,y_pred))

１つ目の文章の言語は ja です
2つ目の文章の言語は en です
3つ目の文章の言語は th です
正解率＝ 1.0


## 同じ文字を使用する言語を分類

#### サンプルデータとして使うWikipedia記事のテキストデータをダウンロードする

In [7]:
import urllib.request as req
# 学習データ用記事
url = "https://github.com/masatokg/sample_photo/raw/master/train.zip"
save_file = "train.zip"
req.urlretrieve(url, save_file)
# テスト検証用データ記事
url = "https://github.com/masatokg/sample_photo/raw/master/test.zip"
save_file = "test.zip"
req.urlretrieve(url, save_file)

# unzip(解凍する)
!unzip -o -q "./train.zip"
print("train.zip解凍")
# unzip(解凍する)
!unzip -o -q "./test.zip"
print("test.zip解凍")

train.zip解凍
test.zip解凍


### ダウンロードしたテキストファイルのうち、学習用を読み込み、学習用の説明変数データと目的データを作る

In [8]:
import glob

# 学習データの準備
index = 0
x_train = [] # 学習用説明変数
y_train = [] #学習用目的変数（ラベル）
for file in glob.glob("./train/*.txt"):# train フォルダのtxt拡張子のファイル分ループ
  # 言語情報のキーワードになるものを文中から取得し、目的変数（ラベル）として設定
  y_train.append( file[8:10] )
  print("ファイルパスの一部をラベルにする:",file, " = ",file[8:10])
  #print(y_train)

  # ファイル内の文字列を連結後、コードポイント頻度を計測し、説明変数として設定
  file_str = ""
  #　ファイルを開いて１行ずつ文字列連結する
  for line in open(file, "r"):
    file_str = (file_str + line)
  # for文終わり
  x_train.append(count_codePoint( file_str ))
#for文終わり
print(x_train)

ファイルパスの一部をラベルにする: ./train/de_cat.txt  =  de
ファイルパスの一部をラベルにする: ./train/en_cat.txt  =  en
ファイルパスの一部をラベルにする: ./train/es_dog.txt  =  es
ファイルパスの一部をラベルにする: ./train/es_elephant.txt  =  es
ファイルパスの一部をラベルにする: ./train/de_elephant.txt  =  de
ファイルパスの一部をラベルにする: ./train/de_dog.txt  =  de
ファイルパスの一部をラベルにする: ./train/en_elephant.txt  =  en
ファイルパスの一部をラベルにする: ./train/es_cat.txt  =  es
ファイルパスの一部をラベルにする: ./train/en_dog.txt  =  en
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]


### 作成した学習データをガウス分布のナイーブベーシズ分類器（GaussianNB）で学習させる

In [9]:
# 学習モデルのインスタンスを生成
clf = GaussianNB()
# 学習させる
clf.fit( x_train, y_train )

GaussianNB(priors=None, var_smoothing=1e-09)

### ダウンロードしたテキストファイルのうち、テスト検証用を読み込み、テスト検証用の説明変数、目的変数データを作る

In [10]:
# 評価（テスト検証）用データの作成
index = 0
x_test = [] # 説明変数
y_test = [] # 目的変数
# tesｔフォルダ内のtxt拡張子のファイルをループする
for file in glob.glob( "./test/*.txt" ):
  # 言語情報になるキーワードをファイルパスから抜き出し、目的変数（ラベルとして設定）
  y_test.append(file[7:9])
  print("ファイルパスの一部をラベルにする:", file, " = ", file[7:9])

  # ファイル内の文字列を連結する
  file_str = ""
  # ファイル内の１行ずつを文字列連結
  for line in open( file, "r" ):
    file_str = file_str + line
  # for文終わり
  # ファイル内の文字列の（Unicodeポイント）の頻度を測定し、説明変数として設定
  x_test.append( count_codePoint( file_str ) )
  #print( file_str ) # ファイル分を表示
  #print( x_test[0] ) # ファイル分を表示
  # for文終わり

ファイルパスの一部をラベルにする: ./test/en_lion.txt  =  en
ファイルパスの一部をラベルにする: ./test/es_lion.txt  =  es
ファイルパスの一部をラベルにする: ./test/de_lion.txt  =  de


### 学習モデルで推論し、評価する

In [11]:
# 推論する
y_pred = clf.predict( x_test )
print( y_pred ) # 推論結果を表示
#　評価する
print( "正解率 = ",accuracy_score( y_test,y_pred ) )

['en' 'es' 'de']
正解率 =  1.0
